In [ ]:
# Remember to delete this before final version
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('utils')
import utils
import tests

# Project 1: An implementation of Schelling's model of segregation in Python

The first project of the course consists of a simplified implementation of a classic example of agent-based modeling: Schelling's model of segreagation. You will implement a simplified version of the model in Python using only custom functions and basic data types. The project is divided in tasks that should be solved in order and build upon each other until a final function that runs an entire simulation of the model. To help making sure that each task is solved correctly before moving on to the next one, we have prepared testing functions that your solution must pass before a task is completed.

This is a long exercise that will require a few hours of work. You are not expected to complete the full project during session hours. In case you don't, you are invited to complete the project in your free time.

Part 1 of this document provides an explanation of Schelling's model. If you're familiar with the model, feel free to skip to part 2 and start solving the project.

## Part 1: An overview of Schelling's model of segregation

The model illustrates how individual behavior and preferences contribute to housing segregation in aggregated results, even when agents (households or individuals) seem to have a strong individual tolerance to be surrounded with other agents they perceive to be in a group different than their own.

The model works as follows: imagine a city is composed by a two-dimensional grid of N x N squares. Only one agent can live in one square, but a square can be empty too and not have a resident agent. The model has an initial (usually random) distribution of agents in the city.

![Small city](img/grid.png)

Take a look at the example above. This is a 7 x 7 city with two groups: "O" and "X". Empty squares represent available lots the agents can potentially move into.

Every location in the city has a "neighborhood" which is composed by the n x n squares surrounding the location in the grid. Every agent in the city has a _similarity threshold_ (_t_), which is the minimum percentage of neighbors of their same group they want to have before being "dissatisfied" with their current location in the city. For simplicity, the value of _t_ is the same for every agent in the city, regardless of the group they belong to.

In the example below, neighborhoods are composed by the 1 x 1 sub-grid surrounding a particular location. Note that empty slots are not considered when calculating the denominator of the similarity score.

![similarity](img/similarity-schelling.png)

When an agent is dissatisfied, they will seek to move to another location that could be a better fit for the minimum similarity they want. The location they will move to is one among all the locations available at the moment that meet the minimum similarity score (_t_). Specifically, they will select the one that is closest to them. If more than acceptable location is closest, then they will go for the one with the highest similarity score.

![moving](img/moving-schelling.png)

The simulation of the model is divided in rounds. When a round starts, the model iterates once through every agent evaluating their satisfaction and moving them to another spot if it applies. A round finishes when all of the agents have been looped through once, executing all the moves that applied. A new round starts when this process is repeated. The simulation finishes when the city achieves convergence (no one moved in an entire round) or a maximum number of rounds have been finalized.

You might note that as a result of the moves in a round some agents that were initially dissatisfied will become satisfied as their neighborhoods change, or vice versa. We will not take care of these cases immediately, but in the next round of the model simulation. The instructions for that part of the implementation will be explained below.

## Part 2: Python implementation

This is how our specific implementation of Schelling's model will work:

1. The initial state of the city (the grid) will be read from pre-defined files
1. When a round of the simulation begins, start the iteration with the square at the top left and perform the following operations:
    - Evaluate if the agent is dissatisfied
    - If they are:
        - Move the agent to its corresponding lot applying the moving rule
        - Update the city
1. Continue with the next square of the row and repeat the process
1. When the rows finishes, continue with the next row
1. When all the rows are looped over, perform the following checks:
    - If no one moved in the round, finish the simulation
    - If more than T rounds have passed, finish the simulation
1. If the simulation didn't finish, continue with a new round of the model

And this is the moving rule we'll use:

- An agent will move to another location that has a higher or equal similarity than the acceptable threshold. If no location has a higher or equal similarity, they will not move
- If more than one location is acceptable, they will move to the closest one in terms of Euclidean distance (formula is below)
- If more than one acceptable location is the closest, they will move to the one that has the highest similarity
- If still more than one is a candidate location, they will be indifferent between them and will choose any

This implementation of the model is divided in a number of tasks that should be completed in order and build upon each other. To solve each task, you should program one or more custom Python functions that execute a particular component of the model and the simulation. Furthermore, we have divided each task into the functions you need to program.

### Task 1: Calculate the similarity score of a location

We've divided this task in two components:

1. Extract the neighborhood of a location
2. Calculate the similarity score of that neighborhood

The neighborhood is extracted using the parameter neighborhood extension (_n_), which determines the size of the a x a surrounding subgrid that encloses a neighborhood. An `n = 1` will return a 3 x 3 subgrid in most casesm and an `n = 3` will return a 7 x 7 subgrid. If a location is in the border of the city, its neighborhood will have a reduced size and it will only "build" on the sides where the city continues.

The image below is an example for `n = 2`. Note that Python starts indices at zero (as opposed to one).

![neighborhoods](img/neighborhoods-schelling.png)

Some tips for this task:

- To debug and try your solution, you can use an example 7 x 7 city we have created for the project. Run the cell below to load it. The city is a list of lists that represents the grids

In [ ]:
city = utils.city('data/city1.csv')

- We have also created a helper function that shows a visualization of a city. Check the example below to use it:

In [ ]:
utils.show_grid(city)

- Since `city` is a list of lists, you can subset a location using concatenated list indexing, as in:

In [ ]:
# Extracting location (0, 6) - upper right corner:
city[0][6]

- For the neighborhood, you will have to subset the list of lists that is the city. You can again use concatenated indexing for this, as in the example below:

In [ ]:
# Neighborhood in rows 1 to 3 and colums 0 to 2
neighborhood = []
for row in city[1:4]:
    neighborhood.append(row[0:3])
utils.show_grid(neighborhood)

- This example can give you an idea of how to extract a neighborhood, but you still need to think of corner cases (literally). For locations near the edges, the functions `min()` and `max()` will be useful to avoid subsetting with indices out of your matrix (city) range
- The similarity score of an agent is equal to the agents of their same group in their neighborhood divided by the total occupied spots. Note that the agent we calculate the similarity for is also counted both in the numerator and the denominator
- For simplicity, if an agent is the only member of their neighborhood, their similarity will be equal to one (100%)

Your solution should be added to the functions below.

In [ ]:
def neighborhood(grid, extension, location):
    
    '''
    Extracts the neighborhood of a location for a specified extension
    
    Inputs:
        - city grid (list of lists): total grid where the neighborhood (subgrid) is extracted from
        - extension (int): the number of adjacent squares to be included in the neighborhood
        - location (list of two int): 
            the row and column index (stating at zero) of the location we extract the neighborhood for.
            examples: [0, 4] or [5, 1]
    Output:
        - neighborhood (list of lists): the neighborhood of this location at the specified extension
    '''
    
    # Your solution goes here
    
    return None

In [ ]:
def similarity(grid, extension, location):
    
    '''
    Calculates the similarity score of the location a city for a specified neighborhood location
    Important: remember to use your function neighborhood() in this function
    
    Inputs:
        - city grid (list of lists): total grid
        - extension (int): the number of adjacent squares to be included in the neighborhood
        - location (list of two int): 
            the row and column index (stating at zero) of the location we calculate the similarity score for
    Output:
        - similarity score (float)
    '''
    
    # Your solution goes here
    
    return None

To test that your solution is correct, we have created the following tester functions. If the cells below return an error message then something needs to be corrected in your solution.

In [ ]:
# Test neighborhood function (to be added)

In [ ]:
# Test similarity score function (to be added)

## Task 2: Evaluate the hypothetical similarity score of an agent if they moved to another location

For task 2 you will take the an agent from a certain location in the grid and evaluate what would be their similarity score if they moved to another location. We'll call this estimation the hypothetical similarity score. Note that this task only consists of one function.

Notes:
- You'll have to use the `neighborhood()` function you defined in task one
- When estimating the hypothetical similarity score, you need to consider that the agent moving will be part of the possible neighborhood. This means that the agent should be counted in the numerator and denominator of the similarity score
- If you take a look at the inputs of the function below, you might be wondering why is the initial location of the agent an input. Keep in mind that we're building a model that is agnostic of the specific values used to differentiate groups 1 and 2, so we need `location1` to know to which group the agent belongs and estimate the similarity score

In [ ]:
def hypothetical_similarity(grid, extension, location1, location2):
    
    '''
    Calculates the similarity score an agent would have in a hypothetical location in the city,
    for a specified neighborhood location
    Important: remember to use your function neighborhood() in this function
    
    Inputs:
        - city grid (list of lists): total grid
        - extension (int): the number of adjacent squares to be included in a neighborhood
        - location1 (list of two int): the row and column index (stating at zero) of the current location of the agent
        - location2 (list of two int): the row and column index (stating at zero) where the agent would be moving
    Output:
        - similarity score (float)
    '''
    
    # Your solution goes here
    
    return None

## Task 3: List all the available and acceptable locations to move for an agent and estimate Euclidean distances


In this task you take a location and a grid to evaluate all the possible acceptable locations that agent could move to. You'll also build a function to estimate Euclidean distances that will come handy in the next task.

We have divided this task in two functions. Note that you might have to use some of your previous functions for some of them.

1. Get the total list of acceptable locations to move
1. Calculate the Euclidean distance between two locations

Notes:

- For the list of acceptable available locations, you'll need to:
    - check that a location is empty
    - estimate the hypothetical similarity the agent will have if they moved there (use the function of task 2)
    - if the result is higher or equal than the acceptable threshold, save that location in the form of `[row index, column index]`
- To calculate the Euclidean distance of two locations, use the following formula:

![euclidean-distance-formula](img/euclidean-distance-formula.png)

- Remember that you can raise a number to a power using the operator `**`

In [ ]:
# Squared root of 2:
2 ** (1/2)

In [ ]:
def list_available(grid, extension, location, threshold):
    
    '''
    Creates a list of all the available locations for an agent to move, where their similarity
    score would be at least equal than their acceptable threshold.
    
    Inputs:
        - city grid (list of lists): total grid
        - extension (int): the number of adjacent squares to be included in a neighborhood
        - location (list of two int): current location of the agent who would move
        - threshold (float): minimum similarity accepted by the agent
    Output:
        - list of available locations that meet the minimum similarity score.
            example: [[4, 3], [0, 2], [6, 6], [0, 8]]
    '''
    
    available = []
    
    # Your solution goes here
    
    return available

In [ ]:
def euclidean_distance(location1, location2):
    
    '''
    Calculates the Euclidean distance between two locations
    
    Inputs:
        - location1 (list of two int): list of row and column indices
        - location2 (list of two int): list of row and column indices
    Output:
        - Euclidean distance (float)
    '''
    
    # Your solution goes here
    
    return None

## Task 4: Move an agent to another location if needed and update the city grid

For this task you'll take one agent and a city grid and evaluate if they're satisfied with their current location. If they are, the grid stays the same. If they're not, you will move the agent to another location using the moving rule and update the city grid accordingly.

We recommend following these steps:

- Evaluate if the agent if satisfied in their current location. If they are, do nothing and return the grid intact
- If they're dissatisfied:
    - check all available locations by running `list_available()`
    - if no location is available, do nothing and return the grid intact
    - if only one is available, change the location of the agent to that one and update the grid
    - if more than one is available, calculate the Euclidean distance to all the potential locations using `euclidean_distance()`
        - if only one is the closest, change the location of the agent to that one and update the grid
        - if more than one location has the same closest proximity, move the agent to the one that has the highest hypothetical similarity (use your function of task 2) and update the grid
        - if more than one location is the closest and has the highest hypothetical similarity, move the agent to any of them (ie: to the first one listed in your available locations) and update the grid

In [ ]:
def move_agent(grid, extension, location, threshold):
    
    '''
    Evaluates if the agent in location is dissatisfied. If they are, the agent is moved to another
    location using the moving rule and the grid is updated.
    
    Inputs:
        - city grid (list of lists): total grid
        - extension (int): the number of adjacent squares to be included in a neighborhood
        - location (list of two int): current location of the agent who is evaluated
        - threshold (float): minimum similarity accepted by the agent
    Output:
        - Updated city grid (list of lists)
    '''
    
    # Your solution goes here
    
    return None

## Task 5: Simulate an entire round of the model

Now that we have a function that evaluates an agent and moves it to another location, we can simulate a round of the model. One round consists of looping through every row and column (in that order) of the city grid, evaluate if the agent there is dissatisfied, and moving them if it applies.

A few notes for this task:
- **Important: Do not loop through the grid itself in your function.** You will need to modify the grid while you're looping and modifying an object while you iterate through its elements will lead to unexpected results and is usually a bad programming practice. Instead, iterate through **the location indices** of the city grid. The difference is as follows:

In [ ]:
# Looping through the grid -- don't do this!
for row in city:
    for element in row:
        # here you're supposed to modify the grid (city). But how, if you're looping through it?
        pass

In [ ]:
# Looping thorugh the location indices -- the correct approach
N = len(city) # number of cols and rows in the grid (remember: our city is a square)

for row_i in range(N):
    for col_i in range(N):
        
        # now row_i and col_i are the current location of the iteration in the grid
        # you can use them to know where you are while you loop. Uncomment the next two lines to see how:
        #location = [row_i, col_i]
        #print(location)
        
        # But more importantly, you can modify the grid as needed now
        pass

- The simulation consists of going through every location of the grid, evaluate if the agent is dissatisfied, and move them if they are. It's basically a spot-by-spot application of your function `move_agent()` that updates the grid on the run
- You'll note that the function below asks you to return two outputs: the grid at the end of the round and the number of times an agent moved in the round. The latter is needed for task 6 (remember that we explained that the simulation will end if no one moves?), that's why we want to keep track of it

In [ ]:
def one_round(grid, extension, threshold):
    
    '''
    Simulates one round of the model in a city grid, given a neighborhood extension and a minimum similarity threshold
    
    Inputs:
        - city grid (list of lists): total grid
        - extension (int): the number of adjacent squares to be included in a neighborhood
        - threshold (float): minimum similarity accepted by an agent
    Outputs:
        - A list with two items:
            - First item is the city grid at the end of the round
            - Second item is the number of moves in the round
    '''
    
    # Your solution goes here
    
    return None

## Task 6: Running the model

Finally, we'll build a function to put it all together and run the model. This function will consecutively simulate rounds of the model until one of these two conditions is met:

- A round has zero moves
- A maximum number of simulation rounds have passed in the model

This function will return a list with two items:

1. The final state of the grid
2. The total number of rounds that were simulated in the run

For your implementation, you will make use of the function `one_round()` that you created in task 5. Remember that this function returns a list of two items, where only the first one is the city. The second item is the number of times agents moved in the round, which you will use to track if a round didn't have a move an terminate the model ahead of time.

Notes:

- This type of function is a great example to use the `while` loop condition. You can associate it to the two conditions of the model, but remember to update the variables the conditions depend on. Otherwise, you might be trapped in an infinite loop! If you think that's happening, you can manually interrupt a cell that is running by clicking in the square (stop) icon at the left side of a cell in Colab
- If you prefer not to use `while`, this function can also be implemented with a `for` loop that is defined with the maximum number of rounds in the model, as in `for round in range(max_rounds)`. In case you find a round with zero moves, you can terminate the loop prematurely and return the result of the function using a syntax similar to this example:

In [ ]:
def loop_million():
    
    for number in range(1000000): # a loop from 0 to 999,999. Should take a long time to run!
        
        if number == 4:
            return True
            # this will terminate the loop in the fifth iteration
            # and will return True

In [ ]:
loop_million()

Your final solution goes below.

In [ ]:
def run_schelling(grid, extension, threshold, max_rounds):
    
    '''
    A function that runs our implementation of Schelling's model of segregation
    
    Inputs:
        - city grid (list of lists): total grid
        - extension (int): the number of adjacent squares to be included in a neighborhood
        - threshold (float): minimum similarity accepted by an agent
    Outputs:
        - A list with two items:
            - first item is the final grid at the end of the simulation
            - second item is the number of rounds that passed in the simulation
    '''
    
    # Your solution goes here
    
    return None